In [7]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import os

# Configurações de Reprodução
np.random.seed(42)
random.seed(42)

In [8]:
print("--- Iniciando Geração da Fato Vendas (MC Supermercados) ---")
print("Carregando tabelas dimensionais...")

# 1. CARREGAMENTO DAS DIMENSÕES (Assumindo que estão na mesma pasta)
try:
    df_produtos = pd.read_csv('dim_produtos.csv')
    df_clientes = pd.read_csv('dim_clientes.csv')
    df_lojas = pd.read_csv('dim_lojas.csv')
    df_promocoes = pd.read_csv('dim_promocoes.csv')
    df_datas = pd.read_csv('dim_datas.csv')
    df_funcionarios = pd.read_csv('dim_funcionarios.csv') # Novo!
    
    # Conversão de datas
    df_promocoes['data_inicio'] = pd.to_datetime(df_promocoes['data_inicio'])
    df_promocoes['data_fim'] = pd.to_datetime(df_promocoes['data_fim'])
    df_datas['data'] = pd.to_datetime(df_datas['data'])
    
    # Filtrar ano 2025
    df_datas_vendas = df_datas[(df_datas['data'] >= '2025-01-01') & (df_datas['data'] <= '2025-12-31')].copy()

except FileNotFoundError as e:
    print(f"ERRO: {e}")
    raise


--- Iniciando Geração da Fato Vendas (MC Supermercados) ---
Carregando tabelas dimensionais...


In [9]:
# --- 2. PREPARAÇÃO DOS DADOS AUXILIARES ---
lista_ids_clientes = df_clientes['id_cliente'].tolist()
lista_skus = df_produtos['sku'].tolist()

# Dicionários de Preço
dict_preco_varejo = df_produtos.set_index('sku')['preco_venda_sup'].to_dict()
dict_preco_atacado = df_produtos.set_index('sku')['preco_venda_atac'].to_dict()
dict_categorias = df_produtos.set_index('sku')['categoria'].to_dict()

# --- MAPEAMENTO DE FUNCIONÁRIOS POR LOJA ---
cargos_permitidos = ['Operador de Caixa', 'Fiscal de Caixa', 'Encarregado', 'Gerente']
df_frente_caixa = df_funcionarios[df_funcionarios['cargo'].isin(cargos_permitidos)]

# Criar dicionário
funcionarios_por_loja = df_frente_caixa.groupby('id_loja')['id_funcionario'].apply(list).to_dict()

# Mapa de Ofertas Semanais (0=Seg, ..., 6=Dom)
mapa_ofertas_semanais = {
    0: 'Higiene/Limpeza', # Segunda
    1: 'Padaria',         # Terça
    2: 'Hortifruti',      # Quarta
    3: 'Açougue',         # Quinta
    4: 'Bebidas'          # Sexta
}

# Perfil de Volume (Multiplicador de movimento)
perfil_lojas = {
    1: {'tipo': 'Varejo',  'fator_movimento': 1.2}, # Matriz
    2: {'tipo': 'Atacado',  'fator_movimento': 0.9}, # Atacado
    3: {'tipo': 'Varejo', 'fator_movimento': 0.7} # Filial Bairro
}

vendas_geral = []
id_venda_global = 1


In [10]:
# --- 3. LOOP PRINCIPAL (DIA A DIA) ---
total_dias = len(df_datas_vendas)
print(f"Gerando vendas para {total_dias} dias...")

for idx, row_data in df_datas_vendas.iterrows():
    data_atual = row_data['data']
    dia_semana = row_data['dia_semana_num']
    eh_final_semana = row_data['eh_final_semana']
    id_data_inteiro = row_data['id_data']
    data_str = data_atual.strftime('%Y-%m-%d')
    
    # A. Checar Promoções (Encartes)
    promocoes_ativas = df_promocoes[
        (df_promocoes['tipo_promocao'] == 'Sazonal (Encarte)') &
        (df_promocoes['data_inicio'] <= data_atual) &
        (df_promocoes['data_fim'] >= data_atual)
    ]
    tem_encarte = len(promocoes_ativas) > 0
    id_promocao_dia = promocoes_ativas['id_promocao'].values[0] if tem_encarte else None
    
    # B. Categoria do Dia (Oferta Recorrente)
    categoria_foco_dia = mapa_ofertas_semanais.get(dia_semana) 
    
    # C. Sazonalidade Geral
    fator_sazonal = 1.0
    if tem_encarte: fator_sazonal += 0.25 # Encarte aumenta fluxo
    if eh_final_semana: fator_sazonal += 0.20 # Fim de semana aumenta fluxo
    if data_atual.day <= 10: fator_sazonal += 0.20 # Início de mês (salário)

    # Loop por Loja
    for id_loja, infos in perfil_lojas.items():
        tipo_loja = infos['tipo']
        fator_loja = infos['fator_movimento']
        
        # Define número de tickets no dia
        base_tickets = 180 
        num_tickets = int(np.random.normal(base_tickets, 30) * fator_sazonal * fator_loja)
        if num_tickets < 50: num_tickets = 50
        
        # Lista de Funcionários DISPONÍVEIS nesta loja
        staff_disponivel = funcionarios_por_loja.get(id_loja, [])
        if not staff_disponivel: staff_disponivel = [1]

        # Gerar Tickets
        for _ in range(num_tickets):
            
            # --- DEFINIÇÃO DO TICKET ---

            # 1. Quem está operando este ticket?
            id_funcionario = random.choice(staff_disponivel)
            
            # 2. Qual PDV (Caixa)? (Simulação simples: lojas tem de 1 a 15 caixas)
            id_caixa = random.randint(1, 15)

            # 3. Cliente Identificado?
            chance_fidelidade = 0.70 if tipo_loja == 'Varejo' else 0.90
            id_cliente = random.choice(lista_ids_clientes) if random.random() < chance_fidelidade else None
            eh_clube = True if id_cliente else False
                
            # 4. Itens
            if tipo_loja == 'Varejo':
                qtd_itens_distintos = random.randint(1, 15) 
            else:
                qtd_itens_distintos = random.randint(5, 30)

            sku_amostra = random.choices(lista_skus, k=qtd_itens_distintos)

            for sku in sku_amostra:
                cat_prod = dict_categorias[sku]
                
                # PREÇO BASE & QUANTIDADE DO ITEM: Varejo ou Atacado?
                if tipo_loja == 'Varejo':
                    preco_base = dict_preco_varejo[sku]
                    qtd = random.choices([1, 2, 3, 4, 5], weights=[60, 20, 10, 5, 5])[0]
                else:
                    preco_base = dict_preco_atacado[sku]
                    qtd = random.choices([1, 6, 12, 24], weights=[10, 40, 30, 20])[0]
                
                # CÁLCULO DE DESCONTOS
                desconto_oferta = 0.0
                desconto_clube = 0.0
                aplica_promo = False

                if categoria_foco_dia and cat_prod == categoria_foco_dia:
                    aplica_promo = True
                elif tem_encarte and random.random() < 0.15: # 15% do mix entra em oferta no encarte
                    aplica_promo = True
                
                if aplica_promo:
                    desconto_oferta = preco_base * random.uniform(0.05, 0.15)
                
                preco_com_oferta = preco_base - desconto_oferta

                if eh_clube and tipo_loja == 'Varejo':
                    desconto_clube = preco_com_oferta * 0.03
                
                valor_final_unit = preco_com_oferta - desconto_clube
                valor_total_linha = valor_final_unit * qtd
                
                # Adicionar registro
                vendas_geral.append({
                    'id_venda': id_venda_global,
                    'id_loja': id_loja,
                    'id_produto': sku,
                    'id_cliente': id_cliente,
                    'id_funcionario': id_funcionario,
                    'id_caixa': id_caixa,
                    'id_data': id_data_inteiro,
                    'data_venda': data_str,
                    'quantidade': qtd,
                    'valor_unitario_tabela': round(preco_base, 2),
                    'desconto_promo_unit': round(desconto_oferta, 2),
                    'desconto_clube_unit': round(desconto_clube, 2),
                    'valor_final_total': round(valor_total_linha, 2),
                    'id_promocao': id_promocao_dia if aplica_promo else None
                })
            
            id_venda_global += 1
            
    # Log de progresso
    if idx % 30 == 0:
        print(f"Processado: {data_str} | Linhas geradas: {len(vendas_geral)}")


Gerando vendas para 365 dias...
Processado: 2025-01-04 | Linhas geradas: 29313
Processado: 2025-02-03 | Linhas geradas: 209892
Processado: 2025-03-05 | Linhas geradas: 409599
Processado: 2025-04-04 | Linhas geradas: 598958
Processado: 2025-05-04 | Linhas geradas: 792002
Processado: 2025-06-03 | Linhas geradas: 982921
Processado: 2025-07-03 | Linhas geradas: 1185291
Processado: 2025-08-02 | Linhas geradas: 1379267
Processado: 2025-09-01 | Linhas geradas: 1566177
Processado: 2025-10-01 | Linhas geradas: 1757999
Processado: 2025-10-31 | Linhas geradas: 1970096
Processado: 2025-11-30 | Linhas geradas: 2176628
Processado: 2025-12-30 | Linhas geradas: 2385746


In [11]:
# --- 4. EXPORTAÇÃO ---
print("Criando DataFrame Final...")
df_fato = pd.DataFrame(vendas_geral)

# LISTA DE COLUNAS CORRIGIDA (Adicionei id_funcionario e id_caixa)
cols = [
    'id_venda', 
    'id_loja', 
    'id_produto', 
    'id_cliente', 
    'id_funcionario', # <--- FALTAVA AQUI
    'id_caixa',       # <--- FALTAVA AQUI
    'id_data', 
    'data_venda', 
    'quantidade', 
    'valor_unitario_tabela', 
    'desconto_promo_unit', 
    'desconto_clube_unit', 
    'valor_final_total', 
    'id_promocao'
]

# Seleciona apenas as colunas desejadas na ordem certa
df_fato = df_fato[cols]

print(f"--- CONCLUÍDO ---")
print(f"Total de Linhas: {len(df_fato)}")
print(f"Total de Vendas (Cupons): {df_fato['id_venda'].nunique()}")


Criando DataFrame Final...
--- CONCLUÍDO ---
Total de Linhas: 2392933
Total de Vendas (Cupons): 217024


In [12]:
# Salvar
df_fato.to_csv('fato_vendas.csv', index=False, encoding='utf-8-sig')
print("Arquivo 'fato_vendas.csv' salvo com sucesso!")

Arquivo 'fato_vendas.csv' salvo com sucesso!
